### Dataset

In [16]:
import pandas as pd
from auto_ts import auto_timeseries
import dill
import talib
import numpy as np
import warnings

warnings.filterwarnings('ignore')

from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam, RMSprop, SGD
from scikeras.wrappers import KerasRegressor
from keras.callbacks import EarlyStopping
from skopt import BayesSearchCV
from sklearn.preprocessing import LabelEncoder
import tensorflow.keras.backend as K
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from keras.losses import categorical_crossentropy
from sklearn.metrics import log_loss
from keras.utils import to_categorical
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold

import h2o
from h2o.automl import H2OAutoML
from h2o.estimators import H2OStackedEnsembleEstimator

In [17]:
# Evito que ciertas columnas se transformen a notacion cientifica en las predicciones
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

In [18]:
columns = [
    # 'Open_time',
    'Open',
    'High',
    'Low',
    # 'Close',
    'Number of trades',
    'Close_BTCUSDT',
    'Volume_BTCUSDT',
    'Number_of_trades_BTCUSDT',
    'Close_ETHUSDT',
    'Volume_ETHUSDT',
    'Number_of_trades_ETHUSDT',
    'Close_BNBUSDT',
    'Volume_BNBUSDT',
    'Number_of_trades_BNBUSDT',
    'SMA_20',
    'EMA_20',
    'Upper_Band',
    'Middle_Band',
    'Lower_Band',
    'RSI',
    'buy_1000x_high_coinbase',
    'sell_1000x_high_coinbase',
    'total_trades_coinbase',	
    'Tweets_Utilizados',
    'Tweets_Utilizados_coin',
    'Tweets_Utilizados_referentes',
    'Tweets_Utilizados_whale_alert',
    'Buy_1000x_high',
    'sell_1000x_high',
    'total_trades_binance'
]

### Armado y entrenamiento de un clasificador a partir de los datos originales

In [19]:
complete_dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv') 
classifier_dataset = complete_dataset[columns]
# classifier_dataset['Open_time'] = pd.to_datetime(classifier_dataset['Open_time'])
classifier_dataset['Tendencia'] = complete_dataset['Tendencia']

clasifier_validation = classifier_dataset[-10:]
classifier_dataset = classifier_dataset[:-10]

In [20]:
display(classifier_dataset.tail())

,Open,High,Low,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Tendencia
941,6.93,7.00,6.70,71088.00,64498.34,31341.46,1375324.00,3155.80,352288.55,861077.00,613.20,453745.52,353114.00,7.43,7.45,9.08,7.43,5.77,38.83,21.00,26.00,33468.00,151,114,0.00,22.00,242.00,219.00,48000.00,Lateral
942,6.86,6.95,6.71,67383.00,63770.01,27085.19,1025561.00,3131.30,252522.65,628635.00,598.00,302119.88,269508.00,7.34,7.38,8.94,7.34,5.74,37.81,29.00,24.00,26619.00,117,106,0.00,14.00,292.00,324.00,42000.00,Lateral
943,6.76,6.87,6.51,64779.00,63461.98,20933.06,912422.00,3255.56,323811.19,734026.00,596.20,268783.91,233820.00,7.24,7.33,8.73,7.24,5.76,38.57,17.00,17.00,25565.00,101,138,0.00,7.00,248.00,179.00,41000.00,Lateral
944,6.81,6.95,6.69,43208.00,63118.62,16949.20,790652.00,3263.45,304766.01,753239.00,600.20,258059.43,206703.00,7.13,7.27,8.38,7.13,5.88,37.66,16.00,20.00,20954.00,82,106,0.00,13.00,173.00,165.00,26000.00,Lateral
945,6.73,6.83,6.47,63006.00,63866.00,28150.23,1152296.00,3216.73,421831.29,943719.00,592.80,330474.01,271926.00,7.03,7.20,8.08,7.03,5.97,36.02,69.00,37.00,33959.00,115,125,0.00,24.00,260.00,188.00,41000.00,Bajista


In [21]:
classifier_dataset.shape

(946, 30)

In [22]:
# X = classifier_dataset.drop(columns=["Tendencia", "Open_time"])
# y = classifier_dataset["Tendencia"]

# y = y.to_numpy().reshape(-1, 1)
# onehot_encoder = OneHotEncoder(sparse=False)
# y_one_hot = onehot_encoder.fit_transform(y)


In [23]:
# display(y_one_hot)

In [24]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,8 mins 40 secs
H2O_cluster_timezone:,America/Argentina/Buenos_Aires
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,2 months and 3 days
H2O_cluster_name:,H2O_from_python_mmarchetta_uvjkm7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.934 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [25]:
h2o_df = h2o.H2OFrame(classifier_dataset)

aml = H2OAutoML(nfolds=5, keep_cross_validation_predictions=True, seed=1234)
aml.train(y='Tendencia', training_frame=h2o_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),6/6
# GBM base models (used / total),1/1
# XGBoost base models (used / total),1/1
# GLM base models (used / total),1/1
# DeepLearning base models (used / total),1/1
# DRF base models (used / total),2/2
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [26]:
leaderboard = aml.leaderboard
print(leaderboard)

model_id                                                   mean_per_class_error    logloss      rmse       mse
StackedEnsemble_BestOfFamily_4_AutoML_2_20240516_184524                0.286811   0.694353  0.481005  0.231366
StackedEnsemble_AllModels_4_AutoML_2_20240516_184524                   0.29174    0.765104  0.5318    0.282811
GLM_1_AutoML_2_20240516_184524                                         0.29387    0.726195  0.49939   0.249391
StackedEnsemble_AllModels_3_AutoML_2_20240516_184524                   0.293986   0.763556  0.531152  0.282122
StackedEnsemble_Best500_1_AutoML_2_20240516_184524                     0.295051   0.757757  0.528731  0.279557
StackedEnsemble_BestOfFamily_3_AutoML_2_20240516_184524                0.295331   0.706258  0.486686  0.236864
StackedEnsemble_AllModels_5_AutoML_2_20240516_184524                   0.296303   0.728034  0.482048  0.23237
StackedEnsemble_BestOfFamily_7_AutoML_2_20240516_184524                0.297298   0.709788  0.49177   0.241838
St

### Guardado de los mejores 5 modelos

In [27]:
top_models = aml.leaderboard.as_data_frame()['model_id'].tolist()[:5]
for i, model_id in enumerate(top_models):
    model = h2o.get_model(model_id)
    file_name = f"modelo_{i}.h2o"
    h2o.save_model(model=model, path=f"automl_classifier", filename=file_name)


### Cargo los mejores 5 modelos

In [28]:
stack_models = []
for i in range(5):
    model_path = f"automl_classifier/modelo_{i}.h2o"
    model = h2o.load_model(model_path)
    # model.nfolds = 5
    stack_models.append(model)

### Armo un ensamble con los mejores 5 modelos (stacks)

In [29]:
# stacked_ensemble = H2OStackedEnsembleEstimator(base_models=stack_models)
# stacked_ensemble.train(y='Tendencia', training_frame=h2o_df)
# h2o.save_model(model=stacked_ensemble, path="automl_classifier", filename="stacked_ensemble_model.h2o")


In [30]:
# stacked_ensemble.model_performance()

In [31]:
# predictions = stacked_ensemble.predict(clasifier_validation)
h2o_validation_df = h2o.H2OFrame(clasifier_validation)
predictions = stack_models[0].predict(h2o_validation_df)
display(predictions)

Parse progress: |

████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,Alcista,Bajista,Lateral
Bajista,0.00959292,0.875568,0.114839
Alcista,0.475926,0.162523,0.361551
Alcista,0.558041,0.0425442,0.399415
Alcista,0.527306,0.0224956,0.450199
Lateral,0.13371,0.328923,0.537367
Lateral,0.36515,0.0724342,0.562416
Bajista,0.0347464,0.705834,0.25942
Lateral,0.123816,0.371275,0.50491
Lateral,0.232675,0.274241,0.493084
Alcista,0.44502,0.126855,0.428126


##### Datos generados por auto ml con prophet:

In [32]:
auto_ml_prophet_df = pd.read_csv('auto_timeseries_models_prophet/predicciones.csv')
h2o_prophet_df = h2o.H2OFrame(auto_ml_prophet_df.drop(columns=["Open_time"]))
auto_mp_prophet_predictions = stack_models[0].predict(h2o_prophet_df)

display(auto_mp_prophet_predictions)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,Alcista,Bajista,Lateral
Bajista,0.00297908,0.897557,0.0994644
Bajista,0.00192641,0.924007,0.0740665
Bajista,0.00134767,0.922917,0.0757357
Bajista,0.0012245,0.926731,0.0720447
Bajista,0.00204072,0.894428,0.103531
Bajista,0.00148705,0.899127,0.0993858
Bajista,0.00104839,0.931915,0.0670368
Bajista,0.00127533,0.925466,0.0732591
Bajista,0.00121564,0.936296,0.0624881
Bajista,0.00092173,0.929034,0.0700445


##### Datos generados por auto ml con stats:

In [33]:
auto_ml_stats_df = pd.read_csv('auto_timeseries_models/predicciones.csv')
h2o_stats_df = h2o.H2OFrame(auto_ml_stats_df.drop(columns=["Open_time"]))
auto_mp_stats_predictions = stack_models[1].predict(h2o_stats_df)

display(auto_mp_stats_predictions)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,Alcista,Bajista,Lateral
Lateral,0.155801,0.26313,0.581069
Lateral,0.147435,0.249894,0.602671
Lateral,0.169648,0.221965,0.608387
Lateral,0.18201,0.234383,0.583607
Lateral,0.227414,0.347254,0.425331
Lateral,0.218574,0.286746,0.494679
Bajista,0.120198,0.505547,0.374255
Bajista,0.119202,0.559704,0.321094
Bajista,0.217946,0.443204,0.338851
Lateral,0.199087,0.384372,0.416541


##### Datos generados por auto ml con modelos clasicos:

In [34]:
auto_ml_df = pd.read_csv('h2o_models/predicciones.csv')
h2o_mp_df = h2o.H2OFrame(auto_ml_df.drop(columns=["Open_time"]))
auto_mp_predictions = stack_models[1].predict(h2o_mp_df)

display(auto_mp_predictions)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,Alcista,Bajista,Lateral
Lateral,0.0757477,0.274241,0.650012
Lateral,0.0923829,0.265074,0.642544
Lateral,0.0816016,0.278412,0.639987
Lateral,0.0879293,0.291449,0.620621
Lateral,0.0918706,0.304325,0.603804
Lateral,0.0966406,0.302377,0.600982
Lateral,0.10146,0.296406,0.602135
Lateral,0.105704,0.356447,0.537848
Lateral,0.1055,0.341313,0.553187
Lateral,0.105553,0.311541,0.582906


##### Datos generados por auto ml con modelos clasicos:

In [35]:
skforecast_df = pd.read_csv('skforecast/predicciones.csv')
skforecast_df = h2o.H2OFrame(skforecast_df)
skforecast_predictions = stack_models[1].predict(skforecast_df)

display(skforecast_predictions)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,Alcista,Bajista,Lateral
Lateral,0.120556,0.34584,0.533604
Lateral,0.0832058,0.307696,0.609098
Lateral,0.0657684,0.410992,0.52324
Bajista,0.0483199,0.489171,0.46251
Bajista,0.042682,0.52159,0.435728
Lateral,0.076882,0.349736,0.573382
Bajista,0.0386812,0.622686,0.338633
Bajista,0.0200539,0.747058,0.232888
Bajista,0.023584,0.674457,0.301959
Bajista,0.0192473,0.597643,0.383109
